In [3]:
import sqlite3
import requests
from bs4 import BeautifulSoup 
from bs4.element import Comment
import random 
import datetime 

In [4]:
#grab visible text on the webpages
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

#find all text using beautiful soup and filter through
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip()for t in visible_texts)

In [5]:
#store all url's from the text document in a list and opens and reads the text document
sites = []
with open("tennisplayerstitles.txt",'r') as infile :
    for line in infile :
        sites.append(line.strip())

In [8]:
#connects to a database I made and either drops or creates a table after running the command
db = sqlite3.connect("tennis.db")
cur = db.cursor()
cur.execute('''DROP TABLE IF EXISTS player_table''')
cur.execute('''CREATE TABLE player_table (player_url TEXT, player_name TEXT, player_bio TEXT, player_born TEXT)''')

In [9]:
#this is grabbing the url, player name, the bio, and the year they were born from the wikipedia pages and creating columns
for site in sites:
    page = requests.get(site)
    soup = BeautifulSoup(page.text, "html.parser")
    name = text_from_html(str(soup.find_all("h1",attrs={"id":"firstHeading"})))
    text = text_from_html(str(soup.find_all("div", attrs={"id":"bodyContent"})))
    born = text_from_html(str(soup.find_all("span", attrs={"class":"bday"})))
    
    
    new_row = [site, name, text, born]
    cur.execute('''INSERT INTO player_table(player_url, player_name, player_bio, player_born) VALUES(?,?,?,?)''', new_row)
    db.commit()

In [10]:
#closing the database
db.close()

In [11]:
%matplotlib inline

import nltk
#from nltk.book import *

import numpy as np
import matplotlib 
import matplotlib.pyplot as plt

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [65]:
#open and read the file to analyze the web scraped text, assign it to a list
file_location = "C:\\Users\\Taylor Toepke\\ADA\\applied-data-analytics\\web-scraping\\"
file_name = "tennis.db"
#file_name = "20170305_michaelpollan_followers.txt"

descs = []
with open(file_location + file_name,'r', encoding= "Latin-1") as ifile :
    next(ifile)
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split(",")
        # spot 6 has the description
        if len(line) >= 7 : # sometimes we don't have descriptions
            descs.extend(line[1].split())
            
        # for now we'll just add on to a big list
#file_location = "C:\\Users\\Taylor Toepke\\ADA\\applied-data-analytics\\web-scraping\\"
#input_file = "tenniswebscrapingproject_TT.txt"



In [66]:
   # spot 6 has the description
if len(descs) >= 7 : # sometimes we don't have descriptions
    sites.extend(line[1].split())
        

In [67]:
#read the file and print some lines
with open(file_location + file_name,'r') as ifile :
    print(ifile.readline(500))


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 3960: character maps to <undefined>

In [68]:
#grab the length of the list
len(descs)

3615

In [69]:
#frequency distribution on the list
tennis_fd = FreqDist(descs)
tennis_fd.most_common(25)

[('to', 197),
 ('Jump', 182),
 ('the', 174),
 ('free', 93),
 ('encyclopedia', 93),
 ('navigation', 91),
 ('search', 91),
 ('in', 51),
 ('CountryÂ', 49),
 ('(sports)', 49),
 ('tennis', 48),
 ('Residence', 45),
 ('.', 40),
 ('player', 38),
 ('a', 34),
 ('of', 33),
 ('and', 30),
 ('is', 30),
 ('This', 28),
 ('living', 28),
 ('For', 24),
 ('name', 23),
 ('for', 22),
 ('Open', 22),
 ('or', 20)]

In [70]:
sum([cnt for w, cnt in tennis_fd])

ValueError: too many values to unpack (expected 2)

In [71]:
#print samples and outcomes of Frequency Distribution 
fd = FreqDist(descs)
fd.most_common(10)
print(fd)

<FreqDist with 1007 samples and 3615 outcomes>


In [72]:
#cleaning the list and importing stopwords
stopwords = set(nltk.corpus.stopwords.words("english"))
stopwords_sp = set(nltk.corpus.stopwords.words("spanish"))

def clean_list(text) :
    ''' takes a list of text and returns a new list with 
        * words cast to lowercase
        * stopwords removed
        * only alphanumeric words
    '''
    text_clean = [w.lower() for w in text if w.isalpha()]
    text_clean = [w for w in text_clean if w not in stopwords]
    text_clean = [w for w in text_clean if w not in stopwords_sp]
    return(text_clean)

In [73]:
descs_clean_gm = clean_list(descs)

In [74]:
#print samples and outcomes
fd_mp = FreqDist(descs_clean_gm)
fd_mp.most_common(20)
print(fd_mp)

<FreqDist with 629 samples and 2150 outcomes>


In [75]:
#print most common clean words
fd_gm = FreqDist(descs_clean_gm)
fd_gm.most_common(50)

[('jump', 182),
 ('free', 93),
 ('encyclopedia', 93),
 ('navigation', 91),
 ('search', 91),
 ('tennis', 51),
 ('countryâ', 49),
 ('residence', 45),
 ('player', 39),
 ('living', 28),
 ('name', 23),
 ('open', 22),
 ('citations', 18),
 ('needs', 16),
 ('additional', 16),
 ('verification', 16),
 ('please', 16),
 ('help', 16),
 ('adding', 16),
 ('reliable', 16),
 ('sources', 16),
 ('material', 16),
 ('unsourced', 16),
 ('australian', 16),
 ('first', 14),
 ('biography', 14),
 ('person', 14),
 ('contentious', 14),
 ('persons', 14),
 ('poorly', 14),
 ('sourced', 14),
 ('must', 14),
 ('removed', 14),
 ('immediately', 14),
 ('united', 13),
 ('states', 13),
 ('article', 12),
 ('final', 12),
 ('wimbledon', 10),
 ('full', 10),
 ('born', 9),
 ('american', 9),
 ('us', 8),
 ('uses', 7),
 ('czech', 7),
 ('australia', 7),
 ('round', 7),
 ('redirects', 6),
 ('family', 6),
 ('french', 6)]

In [78]:
print(descs_clean_gm)

['text', 'free', 'encyclopedia', 'jump', 'navigation', 'jump', 'search', 'redirects', 'uses', 'lost', 'decider', 'wimbledon', 'herbert', 'free', 'encyclopedia', 'jump', 'navigation', 'jump', 'search', 'brazilian', 'tennis', 'player', 'name', 'uses', 'portuguese', 'naming', 'customs', 'first', 'maternal', 'family', 'name', 'pinheiro', 'second', 'paternal', 'family', 'name', 'davi', 'melo', 'marcelo', 'melo', 'countryâ', 'brazil', 'residence', 'belo', 'horizonte', 'free', 'encyclopedia', 'jump', 'navigation', 'jump', 'search', 'french', 'tennis', 'player', 'pop', 'singer', 'yannick', 'noah', 'yannick', 'noah', 'davis', 'countryâ', 'france', 'born', 'may', 'sedan', 'free', 'encyclopedia', 'jump', 'navigation', 'jump', 'search', 'biography', 'living', 'person', 'needs', 'additional', 'citations', 'verification', 'please', 'help', 'adding', 'reliable', 'sources', 'contentious', 'material', 'living', 'persons', 'unsourced', 'poorly', 'sourced', 'must', 'removed', 'immediately', 'free', 'ency